In [1]:
import torch
import brt.nn as nn
from brt.routers.rand import RandomScatterRouter, RandomGatherRouter


class MoE(nn.Module):
    def __init__(self):
        super().__init__()
        self.scatter_router = RandomScatterRouter(dst_num=2)
        self.expert1 = nn.Linear(10, 10)
        self.expert2 = nn.Linear(10, 10)
        self.gather_router = RandomGatherRouter(dst_num=2)

    def forward(self, x):
        route_results = self.scatter_router(x)
        x_0 = self.expert1(route_results[0])
        x_1 = self.expert2(route_results[1])
        x = self.gather_router([x_0, x_1])
        return x


class ThorMoE(nn.Module):
    def __init__(
        self,
    ):
        super().__init__()
        self.moe = MoE()

    def forward(self, x):
        return self.moe(x)


thor_moe = ThorMoE()
thor_moe.cuda()
x = torch.arange(0, 30, dtype=torch.float32).view(3, 10).cuda()
x = thor_moe(x)
print(x)


tensor([[0],
        [1]], device='cuda:0')
tensor([[2]], device='cuda:0')
tensor([[ -1.0387,  -2.1702,   0.5604,  -9.0361,  -1.3278,   0.8976,  -0.7379,
          -1.9866,  -1.0795,  -1.4519],
        [ -2.4891,  -7.4809,   1.4222, -26.0576,  -0.4686,   4.3179,   1.4807,
          -7.0531,  -4.7407,  -6.0141],
        [  5.2827,  13.0791,   2.2238,   3.7662,  -9.0699,  -0.4922,  14.7732,
         -13.0405,  24.5402,  14.3863]], device='cuda:0',
       grad_fn=<AliasBackward0>)
